In [13]:
import os
import warnings
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
#os.environ['CUDA_LAUNCH_BLOCKING'] = str(1)
#os.environ["TORCH_USE_CUDA_DSA"]= str(0)
warnings.filterwarnings('ignore') 


In [14]:
import copy
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset
import torch.optim as optim
from torch.autograd import Variable
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import math
from collections import OrderedDict
import random
from torchsummary import summary
from torchvision import transforms
from torch.utils.data import Dataset
import sys
import torch
import numpy as np
from tqdm import trange
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import mne
from sklearn.preprocessing import StandardScaler

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import import_ipynb

In [16]:
def Accuracy(y_pred, y, train_count):
    max_values, _ = torch.max(y_pred, dim=1, keepdim=True)
    mask = y_pred == max_values
    y_pred = mask.int()
    correct_num = torch.sum(torch.all(torch.eq(y, y_pred), dim=1)).item()
    accuracy = correct_num / train_count
    return accuracy * 100

In [17]:
class MyDataset(Dataset):
    def __init__(self, x, y):
        super(MyDataset, self).__init__()
        self.transforms = transforms.ToTensor()
        self.x = x
        self.y = y
        
    def __getitem__(self, index):
        x = self.x[index, ...]
        y = self.y[index, ...]
        return x, y
    
    def __len__(self):
        return len(self.x)

In [18]:
def Normalize(data):
    normals = []
    scaler = StandardScaler()
    for idx in range(len(data)):
        normals.append(scaler.fit_transform(data[idx]))
    return np.array(normals)

In [19]:
"""
PNES_data = mne.read_epochs(r"D:\MNE Data\PNES\RostamiAlireza.fif", preload=False).get_data(picks='eeg');
TLE_data = mne.read_epochs(r"D:\MNE Data\TLE\AlipoorMohamadHakim.fif", preload=False).get_data(picks='eeg');

PNES_data = torch.Tensor(Normalize(PNES_data)).cuda()
TLE_data = torch.Tensor(Normalize(TLE_data)).cuda()

PNES_labels = torch.Tensor(np.zeros((PNES_data.shape[0]))).cuda()
TLE_labels = torch.Tensor(np.ones((TLE_data.shape[0]))).cuda()

data = torch.cat((PNES_data, TLE_data), axis=0).reshape(-1, 1, 19, 3000)
labels = torch.cat((PNES_labels, TLE_labels))
labels = F.one_hot(labels.to(torch.int64))

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2)
"""

'\nPNES_data = mne.read_epochs(r"D:\\MNE Data\\PNES\\RostamiAlireza.fif", preload=False).get_data(picks=\'eeg\');\nTLE_data = mne.read_epochs(r"D:\\MNE Data\\TLE\\AlipoorMohamadHakim.fif", preload=False).get_data(picks=\'eeg\');\n\nPNES_data = torch.Tensor(Normalize(PNES_data)).cuda()\nTLE_data = torch.Tensor(Normalize(TLE_data)).cuda()\n\nPNES_labels = torch.Tensor(np.zeros((PNES_data.shape[0]))).cuda()\nTLE_labels = torch.Tensor(np.ones((TLE_data.shape[0]))).cuda()\n\ndata = torch.cat((PNES_data, TLE_data), axis=0).reshape(-1, 1, 19, 3000)\nlabels = torch.cat((PNES_labels, TLE_labels))\nlabels = F.one_hot(labels.to(torch.int64))\n\nX_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2)\n'

In [20]:
PNES_data = torch.rand([693, 19, 3000])
TLE_data = torch.rand([693, 19, 3000])
IGE_data = torch.rand([693, 19, 3000])
Focal_data = torch.rand([693, 19, 3000])
PNES_labels = torch.ones([693]) * 2
TLE_labels = torch.ones([693]) * 3
IGE_labels = torch.ones([693])
Focal_labels = torch.ones([693]) * 0

data = torch.cat((PNES_data, TLE_data), axis=0)
data = torch.cat((data, IGE_data), axis=0)
data = torch.cat((data, Focal_data), axis=0).reshape(-1, 1, 19, 3000)


labels = torch.cat((PNES_labels, TLE_labels))
labels = torch.cat((labels, IGE_labels))
labels = torch.cat((labels, Focal_labels))
labels = F.one_hot(labels.to(torch.int64))
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2)

In [21]:
train = MyDataset(X_train, y_train)
train_dataloader = DataLoader(train, batch_size=64, shuffle=True)
test = MyDataset(X_test, y_test)
test_dataloader = DataLoader(test, batch_size=64, shuffle=True)

In [22]:
class net(nn.Module):
    def __init__(self, T, C, input_size, hidden_size, num_layers, spatial_num, dropout, pool):
        super(net, self).__init__()
        
        self.T = T
        self.C = C
        self.spatial_num = spatial_num
        self.dropout = dropout
        self.pool = pool

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.cell_count = self.T // self.input_size

        self.fcn_in = (spatial_num * self.hidden_size)

        self._lstm = nn.LSTM(self.input_size, self.hidden_size, self.num_layers, batch_first=True)

        self.lstm = nn.ModuleList([self._lstm for i in range(self.C)])

        self.cnn_block = nn.Sequential(nn.Conv2d(1, self.spatial_num, (self.C, 1)),
                                       nn.BatchNorm2d(self.spatial_num),
                                       nn.ELU(),



                                       nn.Dropout(self.dropout))

        
        self.fcn = nn.Sequential(nn.Linear(self.fcn_in, 1024), 
                                 nn.ReLU(),
                                 nn.Linear(1024, 16),
                                 nn.ReLU(),
                                 nn.Linear(16, 4))

        #self.fcn = nn.Linear(self.fcn_in, 4)
        self.results = nn.Softmax(dim=1)
    def forward(self, x):
        x = x.reshape(-1, 1, 19, 3000)
        self.N = x.shape[0]
        x = x.reshape(self.N, self.C, self.cell_count, self.input_size)
        _x = None

        for index, cell in enumerate(self.lstm):
            cell_out, _ = cell(x[:, index, :, :], None)
            last_layer_out = cell_out[:, -1, :]
            
            last_layer_out = last_layer_out.unsqueeze(0)
            if _x is None:
                _x = last_layer_out
            else:
                _x = torch.cat((_x, last_layer_out), dim=0)
            

        x = _x.permute(1, 0, 2).unsqueeze(1)

        x = self.cnn_block(x)


        x = x.reshape(self.N, -1)

        x = self.fcn(x)
        x = self.results(x)

        return x

In [23]:
model = net( T = 3000, C = 19, input_size = 60, hidden_size = 30, num_layers=1, spatial_num= 1000, dropout=0.2, pool=1).to(device)
model(X_test.cuda()).shape

torch.Size([555, 4])

In [ ]:
if __name__ == "__main__":
    b = torch.Tensor(479, 3000, 19)
    a = torch.rand(479, 1, 19, 3000).cuda()

    model = net( T = 3000, C = 19, input_size = 60, hidden_size = 30, num_layers=1, spatial_num= 10, dropout=0.2, pool=1).to(device)
    print(model(a).shape)



    criterion = nn.CrossEntropyLoss()
    epochs = 50
    optimizer = optim.Adam(model.parameters(), lr = 1e-3)
    train_count, test_count = X_train.shape[0], X_test.shape[0]



    for epoch in trange(epochs):
        model.train()
        running_loss = 0.0
        correct_num = 0
        batch_size = None

        for index, data in enumerate(train_dataloader):
            x, y = data
            y = y.to(torch.float64)
            batch_size = x.shape[0]
            x, y = x.to(device), y.to(device)
            y_pred = model(x)
            __ = (torch.round(y_pred).to(torch.int64) == y)
            preds = np.all(__.cpu().numpy(), axis=1)
            correct_num += np.count_nonzero(preds)
            loss = criterion(y_pred, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    
        print(f"Train Accuracy:{Accuracy(y_pred, y, y.shape[0])}")

        
    #print(torch.round(model(X_test)))

    #print(y_test)

In [29]:
y.shape

torch.Size([64, 4])